In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### Web Scraping and Acquistion 

**Datasets**

The datasets are acquired from (1) Spotify API (Spotipy) and (2) Billboard Hot 100 2020 

For Spotify, the official "Top Tracks of 2020 Singapore" were chosen as below:

https://open.spotify.com/playlist/37i9dQZF1DX8X0dOzodgNr


For Billboard Hot 100, the data was taken from Wikipedia:

https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2020


**Scraping Playlists from Spotify and Data Wrangling** 

In the following section the dataset from Spotify are acquired, and then undergoes a data wrangling process to get the necessary info on track titles, genre, audio features (danceability, energy,valence,tempo...etc).

Subsequently the lyrics from Genius.com (embedded within Spotify) are also embedded into the dataset. It should be noted some tracks had lyrics that were unavailable.

Python functions were used extensively in this process to get the desired dataset.

In [86]:
### Packages Required to Run this Notebook

##!pip install requests
##!pip install spotipy
##!pip install pip install BeautifulSoup


import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import numpy as np

In [87]:
#CREDENTIALS for Spotify API
cid ='e37f689bb74846d68b1e13ab62a6b036'
secret ='06c86b343635490b87896a7ff0333b28'
# Passing credentials 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [88]:
#FUNCTION TAKES PLAYLIST URL AS ARGUMENT
def get_playlist_data(url):
    uri = []
    track = []
    duration_ms = []
    explicit = []
    track_number = []
    genre =[]
    artist = []
    release_date = []
    popularity=[]
    # search object
    one = sp.playlist_items(url)
    #CREATING PANDAS DATAFRAME
    df1 = pd.DataFrame(one)

    #LOOPING ON EACH VALUE one by one to get info out of it & Appending it to List
    for i,x in df1['items'].items():
        #FOR GENRES
        genre_val = sp.artist(x['track']['artists'][0]['external_urls']['spotify'])
        track_number.append(x['track']['track_number'])
        uri.append(x['track']['uri'])
        release_date.append(x['track']['album']['release_date'])
        track.append(x['track']['name'])
        duration_ms.append(x['track']['duration_ms'])
        explicit.append(x['track']['explicit'])
        one = sp.album_tracks(x['track']['album']['uri'], offset=0, market='SG')
        art = sp.artist_top_tracks(one['items'][0]['artists'][0]['uri'])
        artist.append(art['tracks'][0]['album']['artists'][0]['name'])
        num = int(np.random.randint(0,len(genre_val['genres'])))
        genre.append(genre_val['genres'][num])
        popularity.append(x['track']['popularity'])
    #Creating another pandas dataframe from the list data extracted in above step
    df2 = pd.DataFrame({
    'artist':artist,
    'popularity':popularity,
    'track_uri':uri,
    'track':track,
    'track_number':track_number,
    'genre':genre,
    'release_date':release_date,
    'duration_ms':duration_ms,
    'explicit':explicit,
    'artist':artist})
    
    return df2

In [89]:
#insert output dataframe from the get_album_tracks function
def get_track_info(df):
    danceability = []
    energy = []
    key = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    #Getting audio features from the dataframe extracted in previous step
    for i in df['track_uri']:
        for x in sp.audio_features(tracks=[i]):
            danceability.append(x['danceability'])
            energy.append(x['energy'])
            key.append(x['key'])
            loudness.append(x['loudness'])
            speechiness.append(x['speechiness'])
            acousticness.append(x['acousticness'])
            instrumentalness.append(x['instrumentalness'])
            liveness.append(x['liveness'])
            valence.append(x['valence'])
            tempo.append(x['tempo'])
      #Creating another dataframe      
    df2 = pd.DataFrame({
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo})
    
    return df2

In [90]:
#Merging both dataframe i.e. audio features & Track
def merge_frames(df1, df2):
    df3 = df1.merge(df2, left_index= True, right_index= True)
    return df3

In [91]:
#Scrapping Lyrics from genius.com and combining with the previously fetched data
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    #Requesting genius.com with artist name & song name to get the lyrics
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    #Parsing the HTML data
    html = BeautifulSoup(page.text, 'html.parser')
    #getting the parsed data
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    #RETURN THE SCRAPPED DATA BACK 
    return lyrics

'''
Function Takes Dataframe as argument and then scrapes lyrics from genius.com for respective artist and track
'''
def merge_lyrics(df):
    #Converting artist column to list
    artist_list = df['artist'].to_list()
    #Enumerating track column to get the index values
    for i,track in enumerate(merged_df['track']):
        #Calling the scrap function to scrape the data
        result = scrape_lyrics(artist_list[i], track)
        #If due to some error, result is not coming, don't count that we will handle that in upcoming code
        if result != None:
            df.loc[i, 'lyrics'] = result
        #IF lyrics are not present on genius.com, WRITE DOWN THE UNAVAILABLE MESSAGE
        else:
            df.loc[i, 'lyrics'] = "Unavailable"
    return df

In [92]:
#Sending Playlist Link to get the Data
df1 = get_playlist_data("https://open.spotify.com/playlist/37i9dQZF1DX8X0dOzodgNr")
#Getting Track info i.e. audio features
df2 = get_track_info(df1)
#Merging both Dataframes
merged_df = merge_frames(df1, df2)
#Scraping lyrics and appending it to the dataframe created in above step
spotify_data = merge_lyrics(merged_df)

In [93]:
#WRITING THE DATA TO CSV FILE
spotify_data.to_csv("data/spotify_singapore1.csv",index=False)

In [95]:
## Writing scraped data to sqlite3 database

#!pip install db-sqlite3

import sqlite3

db = sqlite3.connect("data/spotify.db")

spotify_data.to_sql("spotify",db,index = False,if_exists='replace')

**Scraping Billboard Top 100 and Data Wrangling**

Here the dataset for Billboard Top 100 2020 was acquired and wrangled to get the desired dataset.


In [96]:
#GETTING DATA FROM BILLIBOARD
html = requests.get('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2020')

#PARSING THE PAGE using beautiful soup
html_soup = BeautifulSoup(html.text,'html.parser')
table = html_soup.find('table',class_="wikitable")
cols = table.find_all("td")
out = []
for i in cols:
    out.append(i.get_text().strip('""').strip("\n")) 
    
#Extracting out the required fields
title = out[::2]
artist = out[1::2]

#create DataFrame and write output to it
dff = pd.DataFrame({'Title':title,'Artist':artist})

## witing to sqlite3 database

## Connection 

conn = sqlite3.connect("data/billiboard.db")

dff.to_sql("billiboard",conn,index = False,if_exists='replace')##writing to db



#Writing Output to the CSV File

dff.to_csv("data/Billiboard_Data.csv",index=True)

